<a href="https://colab.research.google.com/github/eugenerz/auto-limpieza-notebook/blob/main/ADIP_Formato_de_Limpieza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [155]:

import pandas as pd
import gspread
from google.colab import auth
from google.auth import default
import urllib.parse
import sqlite3
from gspread_dataframe import set_with_dataframe
import numpy as np
import re
import ast
import sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/Limpieza_de_datos')
import mi_libreria as ml
ml.autenticar_drive()
gc = ml.autenticar_google()
conn, cursor = ml.conectar_sqlite_memoria()
from datetime import datetime  # Import the datetime class
import pytz

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [156]:
import pandas as pd

def obtener_nuevos_registros(df_nueva, df_limpia, columna_id='ID_registro'):
    """
    Filtra los registros nuevos que no están en la base limpia.

    Parámetros:
        df_nueva (pd.DataFrame): DataFrame con la base actualizada.
        df_limpia (pd.DataFrame): DataFrame con la base ya limpia.
        columna_id (str): Nombre de la columna de ID único.

    Retorna:
        pd.DataFrame: Subconjunto de df_nueva con solo los registros nuevos.
    """
    ids_limpios = set(df_limpia[columna_id])
    nuevos_registros = df_nueva[~df_nueva[columna_id].isin(ids_limpios)].copy()
    return nuevos_registros

## CARGADO DE BASES


In [157]:

## BASE DE RESPUESTAS
## BASE DE RESPUESTAS
sheet_id = "12sGcwWdXnnwpOVjn2vKlv4SxDLg8rbfe0SsFBQcZx8s"

sheet_name = "Respuestas de formulario 1"

# Encode the sheet name to make it URL-safe
sheet_name_encoded = urllib.parse.quote(sheet_name)

url1 = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name_encoded}"
base_raw=pd.read_csv(url1, encoding='UTF8', dtype=str)


In [158]:

## BASE DE CODIGOS
sheet_id2 = "1RmEEP3bFhHEtT3y2CfSihZDnJRZDXuQFKUCufDqHCWE"
sheet_etl=sheet_id2

sheet_name2 = "Codigos"


url1 = f"https://docs.google.com/spreadsheets/d/{sheet_id2}/gviz/tq?tqx=out:csv&sheet={sheet_name2}"
codigos=pd.read_csv(url1, encoding='UTF8', dtype=str)



In [159]:

## BASE DE CLASIFICACION
sheet_name3 = "Clasificacion"


url1 = f"https://docs.google.com/spreadsheets/d/{sheet_id2}/gviz/tq?tqx=out:csv&sheet={sheet_name3}"
clasificacion_abiertas=pd.read_csv(url1, encoding='UTF8', dtype=str)

## BASE LIMPIA

sheet_name3_1 = "Base Limpia"
sheet_name3_1_encoded = urllib.parse.quote(sheet_name3_1)


url1 = f"https://docs.google.com/spreadsheets/d/{sheet_id2}/gviz/tq?tqx=out:csv&sheet={sheet_name3_1_encoded}"
base_limpia=pd.read_csv(url1, encoding='UTF8', dtype=str)

In [160]:
## Base de ID's


sheet_id4 = "1vAXk01VQsPJIn1WkSem_-NmiuyZKLJa163-JG8vtw2U"
sheet_name4 = "Hoja 1"

sheet_name4_encoded = urllib.parse.quote(sheet_name4)

url1 = f"https://docs.google.com/spreadsheets/d/{sheet_id4}/gviz/tq?tqx=out:csv&sheet={sheet_name4_encoded}"
base_id_equipo=pd.read_csv(url1, encoding='UTF8', dtype=str)

diccionario_correo_a_id = {}
for _, row in base_id_equipo.iterrows():
    diccionario_correo_a_id[row["CORREO"]] = row["ID"]  # Add primary email
    if pd.notna(row["CORREO ALTERNATIVO"]) and row["CORREO ALTERNATIVO"] != '':  # Add alternative email if not empty
        diccionario_correo_a_id[row["CORREO ALTERNATIVO"]] = row["ID"]

In [161]:
# prompt: AÑADE A diccionario_correo_a_id LA SIGUIENTE RELACION 'cervantesfloresj41@gmail.com':'E7'

diccionario_correo_a_id['cervantesfloresj41@gmail.com'] = 'E7'


## FORMATEO DE TITUTLOS Y IDS

In [162]:
base_raw

,Marca temporal,Dirección de correo electrónico,H.INICIO DE ENTREVISTA,COD.Coordinador,M. Seleccione el lugar de la entrevista,S1.¿Se realizará encuesta?,D1. Género \n(ANOTAR SIN PREGUNTAR),D2. ¿Qué edad tiene usted?,D5. ¿En qué Estado vive?,D6. ¿En qué alcaldía vive usted?,...,D4. ¿Cuál es su ocupación principal?,D7. Estado civil,D8. ¿Tiene hijos?,"NSE 01. Pensando en el jefe o jefa de hogar, ¿cuál fue el último año de estudios que aprobó en la escuela?",NSE 02. ¿Cuántos baños completos con regadera y WC (excusado) hay en esta vivienda?,"NSE 03. ¿Cuántos automóviles o camionetas tienen en su hogar, incluyendo camionetas cerradas, con cabina o caja?","NSE 05. De todas las personas de 14 años o más que viven en el hogar, ¿cuántas trabajaron en el último mes?","NSE 06. En esta vivienda, ¿cuántos cuartos se usan para dormir, sin contar pasillo ni baños?",PROTESTA,¿Por qué no se realizó encuesta?
0,8/5/2025 9:02:13,lilianmendozarodriguezcdmx2022@gmail.com,9:00:00 a.m.,C9. MARCO ANTONIO VILLALOBOS HUERTA,EL CAMINERO,2. NO,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3. Rechazó la encuesta
1,8/5/2025 9:06:04,miriamvega.197215@gmail.com,9:03:00 a.m.,C9. MARCO ANTONIO VILLALOBOS HUERTA,EL CAMINERO,1. SI,2. Femenino,19,1.CDMX,12.TLALPAN,...,14. Empleado empresa privada,2. Soltero,2.No,5. Secundaria completa,2. Uno,1. Ninguno,4. Tres,4. Tres,ACEPTO,NaN
2,8/5/2025 9:08:58,cdmxperla@gmail.com,9:07:00 a.m.,C10. JESUS ZUÑIGA ROMAN,COYOACAN,2. NO,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3. Rechazó la encuesta
3,8/5/2025 9:10:09,laboral.bissniness@gmail.com,9:01:00 a.m.,C4. VICTOR MANUEL GUADARRAMA GARCIA,PARQUE HUNDIDO,1. SI,2. Femenino,23,1.CDMX,6.IZTACALCO,...,2. Estudiante,2. Soltero,2.No,11. Licenciatura completa,2. Uno,2. Uno,3. Dos,4. Tres,ACEPTO,NaN
4,8/5/2025 9:11:28,melany080612@gmail.com,9:04:00 a.m.,C4. VICTOR MANUEL GUADARRAMA GARCIA,PARQUE HUNDIDO,1. SI,1. Masculino,25,1.CDMX,7.IZTAPALAPA,...,14. Empleado empresa privada,1. Casado / Unión libre,1.Si,5. Secundaria completa,3. Dos o más,1. Ninguno,3. Dos,3. Dos,ACEPTO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1832,9/5/2025 12:16:06,laboral.bissniness@gmail.com,12:11:00 a.m.,C4. VICTOR MANUEL GUADARRAMA GARCIA,ZÓCALO,1. SI,2. Femenino,48,1.CDMX,5.GUSTAVO A MADERO,...,14. Empleado empresa privada,1. Casado / Unión libre,1.Si,9. Preparatoria completa,2. Uno,2. Uno,2. Una,2. Una,ACEPTO,NaN
1833,9/5/2025 12:16:38,dmtrabajo4@gmail.com,12:16:00 p.m.,C8. ALCIBIADES POLANCO ORTIZ,ALCALDIA XOCHIMILCO,2. NO,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3. Rechazó la encuesta
1834,9/5/2025 12:16:47,miriamvega.197215@gmail.com,12:08:00 a.m.,C1. JORGE MARTIN BERNACHI GARCIA,METRO BUENAVISTA,1. SI,2. Femenino,54,1.CDMX,15.CUAUHTEMOC,...,4. Desempleado,2. Soltero,2.No,9. Preparatoria completa,3. Dos o más,1. Ninguno,1. Ninguna,5. Cuatro o más,ACEPTO,NaN
1835,9/5/2025 12:17:33,cdmxperla@gmail.com,12:07:00 p.m.,C10. JESUS ZUÑIGA ROMAN,ALCALDIA TLÁHUAC,1. SI,1. Masculino,64,1.CDMX,11.TLAHUAC,...,3. Pensionado / Jubilado,1. Casado / Unión libre,1.Si,11. Licenciatura completa,3. Dos o más,3. Dos o más,3. Dos,5. Cuatro o más,ACEPTO,NaN


In [163]:
base=base_raw.copy()

In [164]:
columnas=ml.extraer_headers(base)
base.columns=columnas["Codigo"]
base['index']=base.index
base.insert(0,'index',base.pop('index'))

In [165]:
columnas

,Codigo,Pregunta,Subcodigo
0,Marca temporal,,NaN
1,Dirección de correo electrónico,,NaN
2,H,INICIO DE ENTREVISTA,NaN
3,COD,Coordinador,NaN
4,M,Seleccione el lugar de la entrevista,NaN
...,...,...,...
84,NSE 03,¿Cuántos automóviles o camionetas tienen en s...,NaN
85,NSE 05,De todas las personas de 14 años o más que vi...,NaN
86,NSE 06,"En esta vivienda, ¿cuántos cuartos se usan pa...",NaN
87,PROTESTA,,NaN


In [166]:
base['Marca temporal'] = pd.to_datetime(base['Marca temporal'], errors='coerce')
base['Fecha'] = base['Marca temporal'].dt.date
base['Hora'] = base['Marca temporal'].dt.time

In [167]:
base.insert(1,'Fecha',base.pop('Fecha'))
base.insert(2,'Hora',base.pop('Hora'))
base['Hora'] = base['Hora'].astype(str).str[:5]

In [168]:
base["Marca temporal"] = pd.to_datetime(base["Marca temporal"], errors="coerce")
base["ID_registro"] = base["Marca temporal"].dt.strftime('%Y%m%d%H%M%S')
base.insert(1, 'ID_registro', base.pop('ID_registro'))

In [169]:

base['ID_registro'] = base['ID_registro'] + '_' + base.index.astype(str)


In [170]:
ml.validar_ids_unicos(base, "ID_registro")

✅ Todos los valores en la columna 'ID_registro' son únicos.


## Limpieza

In [171]:
print(base['D3'].value_counts)  # Print unique values in 'D3'

<bound method IndexOpsMixin.value_counts of 0                                       NaN
1            6. Preparatoria o bachillerato
2                                       NaN
3                            7. Profesional
4                             3. Secundaria
                       ...                 
1832    5. Carrera técnica con bachillerato
1833                                    NaN
1834         6. Preparatoria o bachillerato
1835                         7. Profesional
1836                                    NaN
Name: D3, Length: 1837, dtype: object>


In [172]:
base['Dirección de correo electrónico']=base['Dirección de correo electrónico'].map(diccionario_correo_a_id).fillna(base['Dirección de correo electrónico'])
base.rename(columns={'Dirección de correo electrónico': 'ID Encuestador'}, inplace=True)

In [173]:
codigos['Tipo de Pregunta'].value_counts()

,count
Tipo de Pregunta,
MULTIPLE,79
INDICADORES,4
ABIERTA,2
ABIERTA (NO CLASIFICABLE),1
DE REFERENCIA,1


In [174]:

# Filtra las filas donde 'Tipo de Pregunta' es 'MULTIPLE' o 'DE REFERENCIA'
multiple_reference_rows = codigos[codigos['Tipo de Pregunta'].isin(['MULTIPLE', 'DE REFERENCIA'])]

# Obtiene los códigos de esas filas
columnas_a_procesar = multiple_reference_rows['Codigo'].tolist()



In [175]:
base.replace("nan", "", inplace=True)
base.replace("Nan", "", inplace=True)
base.replace("NAN", "", inplace=True)
base.replace(np.nan, "", inplace=True)

In [176]:
base

Codigo,index,ID_registro,Fecha,Hora,Marca temporal,ID Encuestador,H,COD,M,S1,...,D4,D7,D8,NSE 01,NSE 02,NSE 03,NSE 05,NSE 06,PROTESTA,¿Por qué no se realizó encuesta?
0,0,20250805090213_0,2025-08-05,09:02,2025-08-05 09:02:13,E26,9:00:00 a.m.,C9. MARCO ANTONIO VILLALOBOS HUERTA,EL CAMINERO,2. NO,...,,,,,,,,,,3. Rechazó la encuesta
1,1,20250805090604_1,2025-08-05,09:06,2025-08-05 09:06:04,E40,9:03:00 a.m.,C9. MARCO ANTONIO VILLALOBOS HUERTA,EL CAMINERO,1. SI,...,14. Empleado empresa privada,2. Soltero,2.No,5. Secundaria completa,2. Uno,1. Ninguno,4. Tres,4. Tres,ACEPTO,
2,2,20250805090858_2,2025-08-05,09:08,2025-08-05 09:08:58,E35,9:07:00 a.m.,C10. JESUS ZUÑIGA ROMAN,COYOACAN,2. NO,...,,,,,,,,,,3. Rechazó la encuesta
3,3,20250805091009_3,2025-08-05,09:10,2025-08-05 09:10:09,E8,9:01:00 a.m.,C4. VICTOR MANUEL GUADARRAMA GARCIA,PARQUE HUNDIDO,1. SI,...,2. Estudiante,2. Soltero,2.No,11. Licenciatura completa,2. Uno,2. Uno,3. Dos,4. Tres,ACEPTO,
4,4,20250805091128_4,2025-08-05,09:11,2025-08-05 09:11:28,E38,9:04:00 a.m.,C4. VICTOR MANUEL GUADARRAMA GARCIA,PARQUE HUNDIDO,1. SI,...,14. Empleado empresa privada,1. Casado / Unión libre,1.Si,5. Secundaria completa,3. Dos o más,1. Ninguno,3. Dos,3. Dos,ACEPTO,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1832,1832,20250905121606_1832,2025-09-05,12:16,2025-09-05 12:16:06,E8,12:11:00 a.m.,C4. VICTOR MANUEL GUADARRAMA GARCIA,ZÓCALO,1. SI,...,14. Empleado empresa privada,1. Casado / Unión libre,1.Si,9. Preparatoria completa,2. Uno,2. Uno,2. Una,2. Una,ACEPTO,
1833,1833,20250905121638_1833,2025-09-05,12:16,2025-09-05 12:16:38,E10,12:16:00 p.m.,C8. ALCIBIADES POLANCO ORTIZ,ALCALDIA XOCHIMILCO,2. NO,...,,,,,,,,,,3. Rechazó la encuesta
1834,1834,20250905121647_1834,2025-09-05,12:16,2025-09-05 12:16:47,E40,12:08:00 a.m.,C1. JORGE MARTIN BERNACHI GARCIA,METRO BUENAVISTA,1. SI,...,4. Desempleado,2. Soltero,2.No,9. Preparatoria completa,3. Dos o más,1. Ninguno,1. Ninguna,5. Cuatro o más,ACEPTO,
1835,1835,20250905121733_1835,2025-09-05,12:17,2025-09-05 12:17:33,E35,12:07:00 p.m.,C10. JESUS ZUÑIGA ROMAN,ALCALDIA TLÁHUAC,1. SI,...,3. Pensionado / Jubilado,1. Casado / Unión libre,1.Si,11. Licenciatura completa,3. Dos o más,3. Dos o más,3. Dos,5. Cuatro o más,ACEPTO,


In [177]:
## extraccion de codigos de repsuesta en multiples
for col in columnas_a_procesar:
    base[col] = base[col].apply(lambda x: ml.extraer_antes_del_punto(x))


In [178]:
base

Codigo,index,ID_registro,Fecha,Hora,Marca temporal,ID Encuestador,H,COD,M,S1,...,D4,D7,D8,NSE 01,NSE 02,NSE 03,NSE 05,NSE 06,PROTESTA,¿Por qué no se realizó encuesta?
0,0,20250805090213_0,2025-08-05,09:02,2025-08-05 09:02:13,E26,9:00:00 a.m.,C9,EL CAMINERO,2,...,,,,,,,,,,3. Rechazó la encuesta
1,1,20250805090604_1,2025-08-05,09:06,2025-08-05 09:06:04,E40,9:03:00 a.m.,C9,EL CAMINERO,1,...,14,2,2,5,2,1,4,4,ACEPTO,
2,2,20250805090858_2,2025-08-05,09:08,2025-08-05 09:08:58,E35,9:07:00 a.m.,C10,COYOACAN,2,...,,,,,,,,,,3. Rechazó la encuesta
3,3,20250805091009_3,2025-08-05,09:10,2025-08-05 09:10:09,E8,9:01:00 a.m.,C4,PARQUE HUNDIDO,1,...,2,2,2,11,2,2,3,4,ACEPTO,
4,4,20250805091128_4,2025-08-05,09:11,2025-08-05 09:11:28,E38,9:04:00 a.m.,C4,PARQUE HUNDIDO,1,...,14,1,1,5,3,1,3,3,ACEPTO,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1832,1832,20250905121606_1832,2025-09-05,12:16,2025-09-05 12:16:06,E8,12:11:00 a.m.,C4,ZÓCALO,1,...,14,1,1,9,2,2,2,2,ACEPTO,
1833,1833,20250905121638_1833,2025-09-05,12:16,2025-09-05 12:16:38,E10,12:16:00 p.m.,C8,ALCALDIA XOCHIMILCO,2,...,,,,,,,,,,3. Rechazó la encuesta
1834,1834,20250905121647_1834,2025-09-05,12:16,2025-09-05 12:16:47,E40,12:08:00 a.m.,C1,METRO BUENAVISTA,1,...,4,2,2,9,3,1,1,5,ACEPTO,
1835,1835,20250905121733_1835,2025-09-05,12:17,2025-09-05 12:17:33,E35,12:07:00 p.m.,C10,ALCALDIA TLÁHUAC,1,...,3,1,1,11,3,3,3,5,ACEPTO,


In [179]:
base

Codigo,index,ID_registro,Fecha,Hora,Marca temporal,ID Encuestador,H,COD,M,S1,...,D4,D7,D8,NSE 01,NSE 02,NSE 03,NSE 05,NSE 06,PROTESTA,¿Por qué no se realizó encuesta?
0,0,20250805090213_0,2025-08-05,09:02,2025-08-05 09:02:13,E26,9:00:00 a.m.,C9,EL CAMINERO,2,...,,,,,,,,,,3. Rechazó la encuesta
1,1,20250805090604_1,2025-08-05,09:06,2025-08-05 09:06:04,E40,9:03:00 a.m.,C9,EL CAMINERO,1,...,14,2,2,5,2,1,4,4,ACEPTO,
2,2,20250805090858_2,2025-08-05,09:08,2025-08-05 09:08:58,E35,9:07:00 a.m.,C10,COYOACAN,2,...,,,,,,,,,,3. Rechazó la encuesta
3,3,20250805091009_3,2025-08-05,09:10,2025-08-05 09:10:09,E8,9:01:00 a.m.,C4,PARQUE HUNDIDO,1,...,2,2,2,11,2,2,3,4,ACEPTO,
4,4,20250805091128_4,2025-08-05,09:11,2025-08-05 09:11:28,E38,9:04:00 a.m.,C4,PARQUE HUNDIDO,1,...,14,1,1,5,3,1,3,3,ACEPTO,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1832,1832,20250905121606_1832,2025-09-05,12:16,2025-09-05 12:16:06,E8,12:11:00 a.m.,C4,ZÓCALO,1,...,14,1,1,9,2,2,2,2,ACEPTO,
1833,1833,20250905121638_1833,2025-09-05,12:16,2025-09-05 12:16:38,E10,12:16:00 p.m.,C8,ALCALDIA XOCHIMILCO,2,...,,,,,,,,,,3. Rechazó la encuesta
1834,1834,20250905121647_1834,2025-09-05,12:16,2025-09-05 12:16:47,E40,12:08:00 a.m.,C1,METRO BUENAVISTA,1,...,4,2,2,9,3,1,1,5,ACEPTO,
1835,1835,20250905121733_1835,2025-09-05,12:17,2025-09-05 12:17:33,E35,12:07:00 p.m.,C10,ALCALDIA TLÁHUAC,1,...,3,1,1,11,3,3,3,5,ACEPTO,


In [180]:


if 'ID_registro' in base_limpia.columns:
  base_updated = obtener_nuevos_registros(base, base_limpia, columna_id='ID_registro')
  new_rows=len(base_updated)
else:
  base_updated = base
  new_rows=len(base)

mensaje = f"Se han añadido {new_rows} REGISTROS NUEVOS"


In [181]:
try:
    ml.validar_ids_unicos(base, "ID_registro")
    ml.agregar_dataframe_a_hoja(sheet_id2,base_updated,"Base Limpia",gc)
    ml.registrar_ejecucion(sheet_etl,"Base de depuración actualizada",mensaje=mensaje, gc=gc,datetime=datetime,pytz=pytz)
except Exception as e:
    ml.registrar_ejecucion(sheet_etl,"Error de cargado de datos", gc=gc,datetime=datetime,pytz=pytz)
    raise  # Para que el workflow falle igual

✅ Todos los valores en la columna 'ID_registro' son únicos.
DataFrame agregado a partir de la fila 1702 en la hoja 'Base Limpia'.
Registro agregado en fila 29


## EXTRACION DE PREGUNTAS ABIERTAS

In [182]:
try:
    del codigos_abiertas
except NameError:
    pass  # Ignore if the variable doesn't exist



In [183]:
# Filtra las filas donde 'Tipo de Pregunta' es 'MULTIPLE' o 'DE REFERENCIA'
multiple_reference_rows = codigos[codigos['Tipo de Pregunta'].isin(['ABIERTA', 'DE REFERENCIA'])]

# Obtiene los códigos de esas filas
codigos_abiertas = multiple_reference_rows['Codigo'].tolist()

In [184]:
codigos_abiertas.append('Fecha')
codigos_abiertas.append('Hora')
codigos_abiertas.append('index')
codigos_abiertas.append('ID_registro')

In [185]:
base_abiertas=base[codigos_abiertas]

In [186]:
base_abiertas

Codigo,ADIP9,ADIP9_1,ADIP21,Fecha,Hora,index,ID_registro
0,,,,2025-08-05,09:02,0,20250805090213_0
1,4,Más rápido y fácil,,2025-08-05,09:06,1,20250805090604_1
2,,,,2025-08-05,09:08,2,20250805090858_2
3,2,Es menos engorroso,El tiempo de espera,2025-08-05,09:10,3,20250805091009_3
4,1,Es mejor y menos perdida de tiempo,,2025-08-05,09:11,4,20250805091128_4
...,...,...,...,...,...,...,...
1832,2,Evito las colas,,2025-09-05,12:16,1832,20250905121606_1832
1833,,,,2025-09-05,12:16,1833,20250905121638_1833
1834,1,Así me aseguro que el trámite este,No entiendo,2025-09-05,12:16,1834,20250905121647_1834
1835,9,DEPENDE EL TRÁMITE,,2025-09-05,12:17,1835,20250905121733_1835


In [187]:
# prompt: añadir una columna vacia a lado de cada columna en base_abiertas cuyo Tipo de pregunta en codigos sea ABIERTA, el nombre de la columna debe ser del formato columna_clas

for col in base_abiertas.columns:
    tipo_pregunta = codigos[codigos['Codigo'] == col]['Tipo de Pregunta'].iloc[0] if col in codigos['Codigo'].values else None
    if tipo_pregunta == 'ABIERTA':
        nueva_col = col + '_clas'
        base_abiertas.insert(base_abiertas.columns.get_loc(col) + 1, nueva_col, '')


In [188]:


if 'ID_registro' in base_limpia.columns:
  base_abiertas_updated=obtener_nuevos_registros(base_abiertas, clasificacion_abiertas, columna_id='ID_registro')
  new_rows=len(base_abiertas_updated)
else:
  base_abiertas_updated = base_abiertas
  new_rows=len(base_abiertas)

mensaje = f"Se han añadido {new_rows} REGISTROS NUEVOS"


In [189]:
new_rows

137

In [190]:
gc = ml.autenticar_google()
#ml.agregar_dataframe_a_hoja(sheet_id2,base_abiertas_updated,"Clasificacion", gc=gc)

In [191]:
try:
    ml.validar_ids_unicos(base, "ID_registro")
    ml.agregar_dataframe_a_hoja(sheet_id2,base_abiertas_updated,"Clasificacion",gc)
    ml.registrar_ejecucion(sheet_etl,"Base de clasificacion actualizada",mensaje=mensaje, gc=gc,datetime=datetime,pytz=pytz)
except Exception as e:
    ml.registrar_ejecucion(sheet_etl,"Error de cargado de datos", gc=gc,datetime=datetime,pytz=pytz)
    raise  # Para que el workflow falle igual

✅ Todos los valores en la columna 'ID_registro' son únicos.
DataFrame agregado a partir de la fila 1702 en la hoja 'Clasificacion'.
Registro agregado en fila 30
